In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly
import datetime

from scipy.optimize import curve_fit
from scipy.integrate import odeint
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots

init_notebook_mode(connected=True)

# Covid 19 (UPDATED at 2020-04-01 21:33)

Numbers or data without units, errors/uncertainties and context are at best useless and at worst damaging. I mention this because the data presented here is taken as is and has some context people should be aware of. For example let's say the confirmed number of cases for two similarly sized countries A and B are 1000 and 500 cases, is A twice as bad as B? It all depends maybe A is testing twice as many people or B's test not as accurate or many other factors? These factors need to be taken into account in a real analysis I've done this for my own interests, to investigate the data and try out using [Plotly](https://plotly.com/python/) so always take figures and advice from official sources.   


## Raw Data 

Data being collated by Johns Hopkins University and available from this github [repo](https://github.com/CSSEGISandData/COVID-19). So lets plot the raw data for the confirmed cases with linear (left) logarithmic (right) scales.

In [2]:
countries = ['Ireland', 'Spain', 'Italy', 'France', 'Korea, South']#e, 'United Kingdom', 'US', 'Korea, South', 'Germany']
case_thres = 10
colors = plotly.colors.qualitative.Safe

populations = {
    'Ireland': 4.93,
    'Spain': 46.75,
    'Italy': 60.46,
    'France': 65.27,
    'Korea, South': 51.26,
    'United Kingdom': 67.88,
    'US': 331.00,
}
update_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')

In [3]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')


In [4]:
casess_by_country = cases.groupby(cases.columns[1])[cases.columns[4:]].sum()
deaths_by_country = deaths.groupby(deaths.columns[1])[deaths.columns[4:]].sum()

casess_by_country.columns = pd.to_datetime(casess_by_country.columns)
deaths_by_country.columns = pd.to_datetime(deaths_by_country.columns)

In [5]:
fig = make_subplots(rows=1, cols=2, shared_xaxes='all')

colors = plotly.colors.qualitative.Safe
i=0
for country in countries:
        fig.add_trace(go.Scatter(x=casess_by_country.loc[country].index, y=casess_by_country.loc[country], name=country, legendgroup=country, mode='lines+markers',
                                 line={'color':colors[i]}), row=1, col=1)
        fig.add_trace(go.Scatter(x=casess_by_country.loc[country].index, y=casess_by_country.loc[country], name=country, legendgroup=country, mode='lines+markers',
                                 line={'color':colors[i]}, showlegend=False), row=1, col=2)
        i+=1

fig.update_xaxes(title_text=f'Date', row=1, col=1)
fig.update_yaxes(title_text='Cases', row=1, col=1)


fig.update_xaxes(title_text=f'Date', row=1, col=2)
fig.update_yaxes(title_text='Cases', type='log', row=1, col=2)

fig.update_layout(title_text=f"Confirmed Cases (Updated {update_time})")

iplot(fig)

There are a couple of things to notice first while it's easier to differentiate between the data in the log plot the linear plot really indicates the scale also the order of magnitude difference in number of cases this is partially due to their being more cases in some countries but also the population size. The last point is related to the difference in populations in the countries and we can make a rough correction for this by diving by the population (in millions) to obtain the following.

In [6]:
fig = make_subplots(rows=1, cols=2, shared_xaxes='all')

colors = plotly.colors.qualitative.Safe
i=0
for country in countries:
        fig.add_trace(go.Scatter(x=casess_by_country.loc[country].index, y=casess_by_country.loc[country]/populations[country],
                                 name=country, legendgroup=country, mode='lines+markers',
                                 line={'color':colors[i]}), row=1, col=1)
        fig.add_trace(go.Scatter(x=casess_by_country.loc[country].index, y=casess_by_country.loc[country]/populations[country],
                                 name=country, legendgroup=country, mode='lines+markers',
                                 line={'color':colors[i]}, showlegend=False), row=1, col=2)
        i+=1

fig.update_xaxes(title_text=f'Date', row=1, col=1)
fig.update_yaxes(title_text='Cases/Million', row=1, col=1)

fig.update_xaxes(title_text=f'Date', row=1, col=2)
fig.update_yaxes(title_text='Cases/Million', type='log', row=1, col=2)

fig.update_layout(title_text=f"Confirmed Cases Normalised by Population (Updated {update_time})")

iplot(fig)

The differences between countries is much smaller, for the rest of the analysis unless specified I'll be using data normalised by the population. Similar plots can also be made for number of confirmed deaths.

In [7]:
fig = make_subplots(rows=1, cols=2, shared_xaxes='all')

colors = plotly.colors.qualitative.Safe
i=0
for country in countries:
        fig.add_trace(go.Scatter(x=deaths_by_country.loc[country].index, y=deaths_by_country.loc[country]/populations[country],
                                 name=country, legendgroup=country, mode='lines+markers',
                                 line={'color':colors[i]}), row=1, col=1)
        fig.add_trace(go.Scatter(x=deaths_by_country.loc[country].index, y=deaths_by_country.loc[country]/populations[country],
                                 name=country, legendgroup=country, mode='lines+markers',
                                 line={'color':colors[i]}, showlegend=False), row=1, col=2)
        i+=1

fig.update_xaxes(title_text=f'Date', row=1, col=1)
fig.update_yaxes(title_text='Deaths/Million', row=1, col=1)

fig.update_xaxes(title_text=f'Date', row=1, col=2)
fig.update_yaxes(title_text='Deaths/Million', type='log', row=1, col=2)
fig.update_layout(title_text=f"Confirmed Deaths (Updated {update_time})")

iplot(fig)

## Analysis

The spread of infectious diseases, at least in the early stages, is often modeled as an exponential of the form
$$ \frac{N_i}{N_0} = a \exp({\lambda t}) $$
this can be rearranged to obtain
$$log(N_i) = log(a) + log(N_0) + \lambda t $$
or
$$ y = \lambda t + c $$
where $y=log(N_i)$, $c = log(N_0) + log(a)$,$\lambda$ is the growth and the  the doubling time is given by $T_D=log(2)/\lambda$.


We can fit this equation to the data in this case I've chosen to fit the last two weeks or 14 days worth of data. (Clicking on the legend will turn on/off traces)

In [8]:
def get_and_fit(df, country, num_days_fit, num_days_model):
    # Extract and format data
    data = df.loc[country][-num_days_fit-1:]
    x = data.index - data.index[-1]
    y = data.values
    coeffs = np.polyfit(x.days[::-1]*-1 + 1, np.log(y), 1)
    
    fit_y = np.exp(np.polyval(coeffs, list(range(1, num_days_model+1+num_days_fit))))
    fit_x = list(range(-num_days_fit, num_days_model))
    
    return {
        'coeffs' : coeffs,
        'lambda' : np.log(2)/coeffs[0],
        'x': x.days,
        'y': y,
        'fit_y' : fit_y,
        'fit_x' : fit_x
    }

In [9]:
fig = make_subplots(rows=1, cols=2, shared_xaxes='all', shared_yaxes='all')
i=0
for country in countries:
    cbc = get_and_fit(casess_by_country, country, 14, 5)
    dbc = get_and_fit(deaths_by_country, country, 14, 5)
    fig.add_trace(go.Scatter(x=cbc['x'], y=cbc['y'], name=country, legendgroup=country, 
                             mode='lines+markers', line={'color':colors[i]}), row=1, col=1)
    fig.add_trace(go.Scatter(x=cbc['fit_x'], y=cbc['fit_y'], line={'color': 'grey', 'dash': 'longdash'},
                             mode='lines', name=rf'Fit $T_D={cbc["lambda"]:0.2f}, \\ \lambda={cbc["coeffs"][0]:0.2f}$',
                             legendgroup=country),  row=1, col=1)
        
    fig.add_trace(go.Scatter(x=cbc['x'], y=cbc['y'], name=country, legendgroup=country, 
                             mode='lines+markers', line={'color':colors[i]}, showlegend=False), row=1, col=2)
    fig.add_trace(go.Scatter(x=cbc['fit_x'], y=cbc['fit_y'], line={'color': 'grey', 'dash': 'longdash'},
                             mode='lines',
                             legendgroup=country, showlegend=False),  row=1, col=2)
        
#     fig.add_trace(go.Scatter(x=dbc['x'], y=dbc['y'], name=country, legendgroup=country, mode='lines+markers',
#                              line={'color':colors[i]}, showlegend=False), row=1, col=2)
#     fig.add_trace(go.Scatter(x=dbc['fit_x'], y=dbc['fit_y'], line={'color': 'grey', 'dash': 'longdash'},
#                              mode='lines', legendgroup=country, showlegend=False),  row=1, col=2)
    i+=1

fig.update_xaxes(title_text=f'Days', row=1, col=1)
fig.update_yaxes(title_text='Cases/Million', row=1, col=1)

fig.update_xaxes(title_text=f'Days', row=1, col=2)
fig.update_yaxes(title_text='Cases/Million', type='log', row=1, col=2)
fig.update_layout(legend=dict(yanchor='top', xanchor='center'), title_text=f"Confirmed Case Fit (Updated {update_time})")

iplot(fig)

We can also try to compare different countries at similar points along the time line, a simple way to do this is to align the data to some common point, I've chosen to use the point at which the number of confirmed cases exceeded 10.

In [10]:
countries = ['Ireland', 'Spain', 'Italy', 'France', 'Korea, South', 'United Kingdom', 'US']
case_thres = 100
cases_aligned = {}
death_aligned = {}
for country in countries:
        index = casess_by_country.loc[country] > case_thres
        cases_aligned[country] = casess_by_country.loc[country][index].values
        death_aligned[country] = deaths_by_country.loc[country][index].values

In [12]:
fig = make_subplots(rows=2, cols=2, shared_xaxes='all')
i=0
for k, v in cases_aligned.items():
    fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v, name=k, legendgroup=k, mode='lines+markers',
                             line={'color':colors[i]}), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v, name=k, legendgroup=k, mode='lines+markers',
                             line={'color':colors[i]}, showlegend=False), row=1, col=2)
    i+=1
i=0 
for k, v in death_aligned.items():
    fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v, name=k, legendgroup=k, mode='lines+markers',
                             line={'color':colors[i]}, showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v, name=k, legendgroup=k, mode='lines+markers',
                             line={'color':colors[i]}, showlegend=False), row=2, col=2)
    i+=1

fig.update_xaxes(title_text=f'Days (>= {case_thres} cases)', row=1, col=1)
fig.update_yaxes(title_text='Cases',row=1, col=1)
fig.update_xaxes(title_text=f'Days (>= {case_thres} cases)', row=1, col=2)
fig.update_yaxes(title_text='Cases', type='log', row=1, col=2)


fig.update_xaxes(title_text=f'Days (>= {case_thres} cases)', row=2, col=1)
fig.update_yaxes(title_text='Deaths',row=2, col=1)
fig.update_xaxes(title_text=f'Days (>= {case_thres} cases)', row=2, col=2)
fig.update_yaxes(title_text='Deaths', type='log', row=2, col=2)


fig.update_layout(title_text=f"Data Aligned at 100 Confirmed Cases (Updated {update_time})")
iplot(fig)

Again we make the same plots with but normalised by population.

In [13]:
countries = ['Ireland', 'Spain', 'Italy', 'France', 'Korea, South', 'United Kingdom', 'US']
case_thres = 10
cases_aligned = {}
death_aligned = {}
for country in countries:
        index = casess_by_country.loc[country]/populations[country] > case_thres
        cases_aligned[country] = casess_by_country.loc[country][index].values
        death_aligned[country] = deaths_by_country.loc[country][index].values

In [14]:
fig = make_subplots(rows=2, cols=2, shared_xaxes='all')
i=0
for k, v in cases_aligned.items():
    fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v/populations[k], name=k, legendgroup=k, mode='lines+markers',
                             line={'color':colors[i]}), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v/populations[k], name=k, legendgroup=k, mode='lines+markers',
                             line={'color':colors[i]}, showlegend=False), row=1, col=2)
    i+=1
i=0 
for k, v in death_aligned.items():
    fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v/populations[k], name=k, legendgroup=k, mode='lines+markers',
                             line={'color':colors[i]}, showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(len(v)), y=v/populations[k], name=k, legendgroup=k, mode='lines+markers',
                             line={'color':colors[i]}, showlegend=False), row=2, col=2)
    i+=1

fig.update_xaxes(title_text=f'Days (>= {case_thres} cases/Million)', row=1, col=1)
fig.update_yaxes(title_text='Cases/Million',row=1, col=1)
fig.update_xaxes(title_text=f'Days (>= {case_thres} cases/Million)', row=1, col=2)
fig.update_yaxes(title_text='Cases/Million', type='log', row=1, col=2)


fig.update_xaxes(title_text=f'Days (>= {case_thres} deaths/Million)', row=2, col=1)
fig.update_yaxes(title_text='Deaths/Million',row=2, col=1)
fig.update_xaxes(title_text=f'Days (>= {case_thres} deaths/Million)', row=2, col=2)
fig.update_yaxes(title_text='Deaths/Million', type='log', row=2, col=2)
fig.update_layout(title_text=f"Data Aligned at 10 Confirmed Cases/Million (Updated {update_time})")
iplot(fig)

In [11]:
fig = go.Figure()

for country in countries:
    index = casess_by_country.loc[country]/populations[country] > case_thres
    cases_aligned[country] = casess_by_country.loc[country][index].values
    death_aligned[country] = deaths_by_country.loc[country][index].values

    fitable = len(cases_aligned[country]) - 14
    xx = range(1, 15)
    lambdas = []
    for start in range(fitable):
        yy = cases_aligned[country][start:start+14]
        coeffs = np.polyfit(xx, np.log(yy), 1)
        lambdas.append(coeffs[0])


    fig.add_trace(go.Scatter(y=lambdas, name=country))
iplot(fig)
    

In [15]:
def logistic_model(x,a,b,c):
    return c/(1+np.exp(-(x-b)/a))

In [16]:
fig = go.Figure()

for j, country in enumerate(countries):
    i = casess_by_country.loc[country] > 100
    y = casess_by_country.loc[country][i].values
    x = casess_by_country.loc[country][i].index
    x_fit = np.arange(1, len(y)+10)
#     pe = np.polyfit((x-x[0]).days+1, np.log(y), 1)
    pl, sl = curve_fit(logistic_model,(x-x[0]).days+1, y, p0=[3, 20, 10000])
    fig.add_trace(go.Scatter(x=(x-x[0]).days+1, y=y, name=country, mode='lines+markers',
                             line={'color':colors[j]}, legendgroup=country))
    fig.add_trace(go.Scatter(x=x_fit, y=logistic_model(x_fit, *pl), name='Logistic',
                             line={'color': 'grey', 'dash': 'longdash'}, legendgroup=country))
#     fig.add_trace(go.Scatter(x=x_fit, y=np.exp(np.polyval(pe, x_fit)), name='Exp'))

fig.update_layout(yaxis_type='log')
iplot(fig)

In [17]:
countries

['Ireland', 'Spain', 'Italy', 'France', 'Korea, South', 'United Kingdom', 'US']

In [18]:


pl, sl = curve_fit(logistic_model,(x-x[0]).days+1, y, p0=[3, 20, 10000])
pe, pl

NameError: name 'pe' is not defined

In [ ]:
x_fit = np.arange(1, 30)
fig = go.Figure()
fig.add_trace(go.Scatter(x=(x-x[0]).days+1, y=y, name='Ireland'))
fig.add_trace(go.Scatter(x=x_fit, y=logistic_model(x_fit, *pl), name='Logistic'))
fig.add_trace(go.Scatter(x=x_fit, y=np.exp(np.polyval(pe, x_fit)), name='Exp'))
fig.update_layout(yaxis_type='log')

In [ ]:
country = 'Ireland'
i = casess_by_country.loc[country] > 100
y = casess_by_country.loc[country][i].values
x = casess_by_country.loc[country][i].index

xx = ((x - x[0]).days + 1).values
yy = y/populations[country]

In [ ]:
xx, yy

In [ ]:
def sir_model(y, x, beta, gamma):
    S = -beta * y[0] * y[1] / N
    R = gamma * y[1]
    I = -(S + R)
    return S, I, R

def fit_odeint(x, beta, gamma):
    return odeint(sir_model, (S0, I0, R0), x, args=(beta, gamma))[:,1]

N = populations[country]
I0 = yy[0]
S0 = N - I0
R0 = 0.0

x_fit=np.arange(1, 40)

popt, pcov = curve_fit(fit_odeint, xx, yy)
fitted = fit_odeint(x_fit, *popt)
fitted, popt

In [ ]:
odeint(sir_model, (S0, I0, R0), xx, args=(0.1, 0.1))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=xx, y=yy, name='Data'))
fig.add_trace(go.Scatter(x=x_fit, y=fitted, line={'color': 'grey', 'dash': 'longdash'}, name='Fit'))